In [1]:
import os, sys
sys.path.insert(0, '/home/trax/trax/website')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "trax.settings")
import django
django.setup()

In [3]:
from fuzzywuzzy import fuzz, process
from tracks.models import Track
from vehicles.models import Vehicle, VehicleClass
from trax.utils import get_object_by_string

def lt2millis(parts):
    m, rest = parts.split(':')
    millis = 1000 * (int(m)*60 + float(rest))
    return millis

def get_object_by_string(s, klass, attribute, score_cutoff=90):
    choices = klass.objects.all().values_list(attribute, flat=True)
    result = process.extractOne(s, choices,
                                score_cutoff=score_cutoff)
    if result:
        return klass.objects.get(**{attribute:result[0]})


In [19]:
data = '''Sports	9F	1:04.125	119.75
Sports	9F Cabrio	1:04.157	119.75
Sports	Alpha	1:05.732	117.75
Sports	Banshee	1:06.066	117.75
Sports	Bestia GTS	1:04.364	118.75
Sports	Blista Compact	1:15.408	103
Sports	Buffalo	1:08.503	112.25
Sports	Buffalo S	1:08.001	112.25
Sports	Carbonizzare	1:05.699	119.5
Sports	Comet	1:05.130	119.5
Sports	Comet Retro Custom	1:05.632	121.25
Sports	Coquette	1:06.333	119.25
Sports	Drift Tampa	1:08.334	114.25
Sports	Elegy Retro Custom	1:03.483	115.25
Sports	Elegy RH8	1:02.529	118.5
Sports	Feltzer	1:03.229	119.5
Sports	Furore GT	1:08.502	120.25
Sports	Fusilade	1:08.652	117.75
Sports	Futo	1:12.706	119.25
Sports	Go Go Monkey Blista	1:15.402	103
Sports	Jester	1:03.763	118.75
Sports	Jester (Racecar)	1:03.363	119.25
Sports	Khamelion	1:08.301	102.25
Sports	Kuruma	1:06.800	112
Sports	Kuruma (Armored)	1:09.036	109.75
Sports	Lynx	1:07.133	121.5
Sports	Massacro	1:03.864	121.75
Sports	Massacro (Racecar)	1:03.810	121.75
Sports	Omnis	1:07.600	112.5
Sports	Penumbra	1:12.243	105.25
Sports	Rapid GT	1:08.419	119.5
Sports	Rapid GT	1:08.434	119.5
Sports	Raptor	1:11.371	103.25
Sports	Schafter LWB	1:09.769	109.5
Sports	Schafter V12	1:05.121	124.25
Sports	Schwartzer	1:11.105	117
Sports	Seven-70	1:04.764	123.5
Sports	Specter	1:04.398	121.25
Sports	Specter Custom	1:04.998	121.25
Sports	Sprunk Buffalo	1:07.234	116
Sports	Sultan	1:05.834	115.75
Sports	Surano	1:04.531	121
Sports	Tropos Rallye	1:06.866	119.5
Sports	Verlierer	1:05.659	121.75
Sports Classics	Casco	1:10.337	120
Sports Classics	Coquette Classic	1:11.470	118
Sports Classics	Franken Stange	1:16.909	106.5
Sports Classics	JB 700	1:12.071	119.25
Sports Classics	Mamba	1:12.372	117
Sports Classics	Manana	1:23.183	98.25
Sports Classics	Monroe	1:11.172	122
Sports Classics	Peyote	1:24.751	98.25
Sports Classics	Pigalle	1:10.169	121.25
Sports Classics	Roosevelt	1:17.344	98.5
Sports Classics	Roosevelt Valor	1:16.406	98.5
Sports Classics	Stinger	1:13.239	112
Sports Classics	Stinger GT	1:12.972	112
Sports Classics	Stirling GT	1:06.166	112
Sports Classics	Tornado	1:26.353	98.25
Sports Classics	Tornado Cabrio	1:26.356	98.25
Sports Classics	Tornado Custom	1:24.417	98.25
Sports Classics	Tornado Mariachi	1:31.891	91.25
Sports Classics	Tornado Rat Rod	1:18.277	98.75
Sports Classics	Z-Type	1:09.368	126.25
Supers	811	1:03.930	129.25
Supers	Adder	1:04.964	124.75
Supers	Banshee 900R	1:04.565	131
Supers	Bullet	1:08.034	118.75
Supers	Cheetah	1:03.530	120.25
Supers	Entity XF	1:02.323	121.5
Supers	ETR1	1:01.728	120.75
Supers	FMJ	1:01.795	125
Supers	Infernus	1:02.929	117.75
Supers	Itali GTB	1:01.861	126.25
Supers	Itali GTB Custom	1:02.896	127
Supers	Nero	1:01.495	126.5
Supers	Nero Custom	1:01.061	127.25
Supers	Osiris	1:01.394	122
Supers	Penetrator	1:02.562	124
Supers	RE-7B	1:00.627	123.5
Supers	Reaper	1:02.716	121.75
Supers	Rocket Voltic	1:03.144	106
Supers	Sultan RS	1:04.364	117.25
Supers	T20	1:01.287	122.25
Supers	Tempesta	1:00.803	121.25
Supers	Turismo R	1:01.998	121.75
Supers	Tyrus	1:02.228	123.5
Supers	Vacca	1:03.964	120.25
Supers	Voltic	1:05.798	106
Supers	X80 Proto	1:01.175	127.5
Supers	Zentorno	1:00.960	122'''

In [39]:
for line in data.splitlines():
    vdata = line.split('\t')
    if vdata[2]:
        vv = get_object_by_string(vdata[1], Vehicle, "name", score_cutoff=95)
        if not vv:
            print(".............................NOT FOUND:", vdata[1], "CREATING NOW.")
            vv = Vehicle(name=vdata[1])
            # continue
        millis = lt2millis(vdata[2])
        vv.cc_laptime_millis = millis
        vv.topspeed_kmh = float(vdata[3]) * 1.60934
        if (vdata[1] != vv.name):
            print("..... NOT MODIFIED:", vdata[1], vv.name, vv.pk, vv.cc_laptime_millis, vv.topspeed_kmh)
        else:
            vv.save()
            print(vv.name, vv.cc_laptime_millis, vdata[2])

9F 64125.0 1:04.125
9F Cabrio 64157.0 1:04.157
Alpha 65732.0 1:05.732
Banshee 66066.0 1:06.066
Bestia GTS 64364.00000000001 1:04.364
Blista Compact 75408.0 1:15.408
Buffalo 68503.0 1:08.503
Buffalo S 68001.0 1:08.001
Carbonizzare 65699.0 1:05.699
Comet 65129.99999999999 1:05.130
Comet Retro Custom 65632.0 1:05.632
Coquette 66333.0 1:06.333
Drift Tampa 68334.0 1:08.334
Elegy Retro Custom 63483.0 1:03.483
Elegy RH8 62529.0 1:02.529
Feltzer 63229.0 1:03.229
Furore GT 68502.0 1:08.502
Fusilade 68652.0 1:08.652
Futo 72706.0 1:12.706
Go Go Monkey Blista 75402.0 1:15.402
Jester 63763.0 1:03.763
Jester (Racecar) 63363.0 1:03.363
Khamelion 68301.0 1:08.301
Kuruma 66800.0 1:06.800
Kuruma (Armored) 69036.0 1:09.036
Lynx 67133.0 1:07.133
Massacro 63864.0 1:03.864
Massacro (Racecar) 63810.0 1:03.810
Omnis 67600.0 1:07.600
Penumbra 72243.0 1:12.243
Rapid GT 68419.0 1:08.419
Rapid GT 68434.0 1:08.434
Raptor 71371.0 1:11.371
Schafter LWB 69769.0 1:09.769
Schafter V12 65120.99999999999 1:05.121
Schwart

In [71]:
Vehicle.objects.exclude(classes__in=vc)

<QuerySet []>

In [66]:
v = Vehicle.objects.get(name="Ruiner 2000")
v.classes.add(muscle)
v.save()

In [69]:
from vehicles.models import VehicleClass
vc  = list(VehicleClass.objects.all().values_list("pk", flat=True))
muscle = VehicleClass.objects.get(name="Muscle")
off = VehicleClass.objects.get(name="Offroad")
suv = VehicleClass.objects.get(name="SUV")
mo = VehicleClass.objects.get(name="Motorcycle")

In [44]:
vc

[9, 11, 12, 8, 3, 10, 5, 4, 7, 2, 1, 17, 6]

In [63]:
a, = Vehicle.objects.filter(name="Park Ranger")

In [64]:
a.classes.all()

<QuerySet [<VehicleClass: SUV>]>